# Finite Element Error Tutorial
#### Extreme Axial Compression

This example shows how mesh choice and geometrical instability can give unpredictable results in large-deformation scenarios.

In [2]:
import os
import warnings
warnings.simplefilter("always")
from finiteelementanalysis import pre_process as pre
from finiteelementanalysis import pre_process_demo_helper_fcns as pre_demo
from finiteelementanalysis.solver import hyperelastic_solver
from finiteelementanalysis import visualize as viz
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

## Create Geometry and Mesh

In this example, we consider a simple 2 dimensional rectangular beam, meshed rather coarsely along its length.

In [3]:
# for saving files later
tutorials_dir = os.path.abspath('')

# FEA problem info
ele_type = "D2_nn6_tri"
ndof = 2

# Define domain
L = 10.0      # twice length in x-direction
H = 0.2      # height in y-direction
nx = 10       # number of elements in x
ny = 4       # number of elements in y, keep this an even number if you want the analytical solution to be able to compute midline deformation

# Generate mesh
coords, connect = pre.generate_rect_mesh_2d(ele_type, -L/2, -H/2, 0, H/2, nx, ny)

mesh_img_fname = tutorials_dir + "/c-mesh.png"
pre_demo.plot_mesh_2D(str(mesh_img_fname), ele_type, coords, connect)


![mesh image](c-mesh.png "Mesh")

## Specify a Material and Apply Boundary Conditions

After assigning suitable material properties, we apply boundary conditions to compress the beam axially.
We specify that the ends should move towards each other with a speed of 0.7, and should be fixed in the y-direction.

In [4]:
# Choose material properties
material_props = np.array([134.6, 83.33])  # [mu, K]

# Identify boundaries
boundary_nodes, boundary_edges = pre.identify_rect_boundaries(coords, connect, ele_type, -L/2, 0, -H/2, H/2)

# Apply boundary conditions:
# 1. Fix left end boundary
fixed_left = pre.assign_fixed_nodes_rect(boundary_nodes, "left", 0.7, 0)
# 2. Fix right end boundary
fixed_right = pre.assign_fixed_nodes_rect(boundary_nodes, "right", -0.7, 0)
# Combine BCs (assuming the functions return arrays of shape (3, n_bc))
fixed_nodes = np.hstack((fixed_left, fixed_right))

# No distributed load is applied
dload_info = np.empty((ndof + 2, 0))

## Run the Solver

In [5]:
# Number of incremental loading steps
nr_num_steps = 100

# Run the solver
displacements_all, nr_info_all = hyperelastic_solver(
    material_props,
    ele_type,
    coords.T,      # solver expects coords as (ncoord, n_nodes)
    connect.T,     # and connectivity as (n_nodes_per_elem, n_elems)
    fixed_nodes,
    dload_info,
    nr_print=True,
    nr_num_steps=nr_num_steps,
    nr_tol=1e-8,
    nr_maxit=30,
)

final_disp = displacements_all[-1]  # final global displacement vector (length = n_nodes * ndof)

# Analytical solution: For a homogeneous extension,
#   u_x(x) = (lambda_target - 1) * x, and u_y(x) = 0.
# Extract nodes near mid-height to get a 1D slice.
tol_y = H / 20.0  # tolerance for y coordinate
mid_nodes = [i for i in range(coords.shape[0]) if abs(coords[i, 1] - H/2) < tol_y]
mid_nodes = sorted(mid_nodes, key=lambda i: coords[i, 0])  # sort by x-coordinate

# Extract y-coordinates and computed u_y from the final displacement.
x_vals = np.array([coords[i, 0] for i in mid_nodes])
computed_u_y = np.array([final_disp[ndof * i + 1] for i in mid_nodes])

Step 0, load factor = 0.010
Iteration 1, Correction=1.000000e+00, Residual=7.856742e-05, tolerance=1.000000e-08
Iteration 2, Correction=5.228553e-05, Residual=3.524575e-06, tolerance=1.000000e-08
Iteration 3, Correction=8.322217e-09, Residual=8.842394e-12, tolerance=1.000000e-08
Step 1, load factor = 0.020
Iteration 1, Correction=5.000005e-01, Residual=7.856742e-05, tolerance=1.000000e-08
Iteration 2, Correction=3.953569e-05, Residual=3.543158e-06, tolerance=1.000000e-08
Iteration 3, Correction=1.697454e-08, Residual=8.924466e-12, tolerance=1.000000e-08
Iteration 4, Correction=1.255522e-13, Residual=1.820751e-16, tolerance=1.000000e-08
Step 2, load factor = 0.030
Iteration 1, Correction=3.333342e-01, Residual=7.856742e-05, tolerance=1.000000e-08
Iteration 2, Correction=9.852214e-05, Residual=3.561669e-06, tolerance=1.000000e-08
Iteration 3, Correction=1.125453e-07, Residual=8.213655e-12, tolerance=1.000000e-08
Iteration 4, Correction=1.304124e-14, Residual=1.958979e-16, tolerance=1.000

## Plot Results

Using matplotlib and the visualization module, we can create plots and animations showing the beam's behavior.

In [6]:
# Plot the computed u_y vs. x.
plt.figure(figsize=(8, 6))
plt.plot(x_vals, computed_u_y, 'ro-', label="u_x")
plt.xlabel("y (m)")
plt.ylabel("u_y (m)")
plt.title("Final u_y(x)")
plt.legend()
plt.grid(True)
plt.tight_layout()

# Save the plot image to the tutorials directory.
img_fname = tutorials_dir + "/c-final_u_y.png"
plt.savefig(str(img_fname))

# Save an animation of the deformation
img_name = tutorials_dir + "/c-kinking_beam.gif"
gif_displacements = displacements_all
for i_extend in range(20): gif_displacements += [final_disp]
viz.make_deformation_gif(gif_displacements, coords, connect, ele_type, img_name, interval=40)

![Buckling beam animation](c-kinking_beam.gif "FEA failure")

This unexpected buckling failure due to extreme compression seems to be very mesh dependent.
Geometric and numerical instabilities can feed off one another to give such dramatic and nonphysical results, especially when the mesh is too coarse.

![Final beam shape](c-final_u_y.png "Final beam shape")